In [5]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [6]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [8]:
%%sql
SELECT *
FROM genre;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


Which genres are the most popular in the US?

In [34]:
%%sql
WITH usa_tracks_sold AS
    (SELECT il.* FROM 
    invoice_line il
    INNER JOIN invoice i ON il.invoice_id = i.invoice_id
    INNER JOIN customer c ON i.customer_id = c.customer_id
    WHERE c.country = "USA"),
    track_genre AS
    (SELECT g.name genre, t.track_id
    FROM track t
    INNER JOIN genre g ON g.genre_id = t.genre_id)
SELECT tg.genre genre, 
       COUNT(uts.invoice_line_id) tracks_sold,
       (CAST(COUNT(uts.invoice_line_id) AS FLOAT) * 100 / (SELECT COUNT(*) FROM usa_tracks_sold)) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track_genre tg ON tg.track_id = uts.track_id
GROUP BY 1 
ORDER BY 2 DESC

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


**Observations:**
1. Rock is clearly the most lucrative genre in the US.
2. Alternative Punk and Metal are distant second and third but close to each other. 

In [44]:
%%sql
SELECT e.first_name || " " || e.last_name employee_name,
       e.title, 
       e.hire_date hired, SUM(i.total) total_sales
FROM employee e 
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON c.customer_id= i.customer_id
GROUP BY 1;

Done.


employee_name,title,hired,total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.920000000002


Clearly, the sales figures are affected by when a support agend was hired. The earlier, the agent was hired, the higher the sales. It may make sense to look at average sales per month if the information is available or can be computed.

Let's understand trends by country. If a country has only 1 customer, we will classify them under "Other"

In [48]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


We see clearly that lifetime value of customers can vary significantly across countries, India, UK and Czech Republic may present significant opportunity. However, there is very little data, hence, making conclusions is not advisable.